In [ ]:
! pip install uci-dataset
! pip install imblearn

In [ ]:
import uci_dataset as dataset
from imblearn.combine import SMOTETomek
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
# fetch pakinson dataset
parkinsons = dataset.load_parkinson()
X, y = parkinsons.drop(['status', 'name'], axis=1), parkinsons['status']
# Hybridization: SMOTE + Tomek Links
smote_tomek = SMOTETomek(random_state=250)
X_smote_tomek, y_smote_tomek = smote_tomek.fit_resample(X, y)
df_smote_tomek = pd.concat([X_smote_tomek, pd.Series(y_smote_tomek)], axis=1)
# Declare features and target
X, y = df_smote_tomek.drop(['status'], axis=1), df_smote_tomek['status']
# Splite dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Data preprocess and model construction
_sc = StandardScaler()
_pca = PCA(n_components=0.85)
_model = RandomForestClassifier(n_estimators=20, random_state=100)
random_forest_model = Pipeline([
    ('std_scaler', _sc),
    ('pca', _pca),
    ('classifier', _model)
])

# Training
random_forest_model.fit(X_train, y_train)

# Evaluation and prediction
acc= random_forest_model.score(X_test, y_test)
y_pred = random_forest_model.predict(X_test)

# Classification Report
reports = classification_report(y_test, y_pred)

In [ ]:
import pickle
pickle.dump(random_forest_model, open('PD_predictor.pkl','wb'))

In [ ]:
model = pickle.load(open('PD_predictor.pkl', 'rb'))

In [ ]:
model.score(X_test, y_test)

0.9661016949152542